In [1]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5975 sha256=d874c66bb17ac03c0fdee4d3a350b91a163fe5950524f19293b95b146010f76c
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\1b\02\6c\a45230be8603bd95c0a51cd2b289aefdd860c1a100eab73661
Successfully built apyori
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
df=pd.read_csv('민원.csv', encoding='cp949')
df=df.drop(['가상번호','등록일자','처리상태'], axis=1)
df=df[df['제목'].isnull()==False]

In [2]:
import re
def text_clearning(text):
    hangul = re.compile('[^ㄱ-ㅣ가-힣]+')
    result = hangul.sub('',text)
    return result
df['제목']=df['제목'].apply(lambda x: text_clearning(x))

In [3]:

from konlpy.tag import Okt
from collections import Counter

# 한국어 불용어 사전
korean_stopwords_path = 'korean_stopwords.txt'
with open(korean_stopwords_path, encoding='utf-8') as f:
    stopwords=f.readlines()
stopwords=[x.strip() for x in stopwords]

def get_nouns(x):
    nouns_tagger=Okt()
    nouns = nouns_tagger.nouns(x)
    nouns = [noun for noun in nouns if len(noun)>1]
    nouns = [noun for noun in nouns if noun not in stopwords]
    return nouns



In [4]:
df['nouns']=df['제목'].apply(lambda x: get_nouns(x))

In [5]:
df

,분류,제목,nouns
0,건설ㆍ교통,스쿨존불법오토바이,"[스쿨존, 불법, 오토바이]"
1,건설ㆍ교통,건설행정과직원들의불합리한단속을신고합니다,"[건설, 행정, 직원, 불합리, 단속, 신고]"
2,건설ㆍ교통,동명동인도공사인가주차장공사인가,"[동명동, 공사, 주차장, 공사]"
3,건설ㆍ교통,동명동인도공사인가주차장공사인가,"[동명동, 공사, 주차장, 공사]"
4,건설ㆍ교통,동명동인도주차위험,"[동명동, 인도, 주차, 위험]"
...,...,...,...
4283,기타,남광주시장천변도로점검,"[광주시, 장천, 변도, 점검]"
4285,기타,식중독단체감염,"[식중독, 단체, 감염]"
4287,건설ㆍ교통,푸른길전동킥보드규제,"[보드, 규제]"
4289,환경ㆍ청소,대형할인점쇼케이스콤프실소음과진동조치,"[대형, 할인점, 쇼케이스, 콤프실, 소음, 진동, 조치]"


In [6]:
# 연관 키워드 추출
#!pip install apriori apyori

In [7]:
from apyori import apriori

transactions=df['nouns'].tolist()

#공백문자열 방지
transactions=[transaction for transaction in transactions if transactions]
 

In [8]:
results= list(apriori(transactions, 
                      min_support=0.1, 
                      min_confidence=0.2,
                     min_list=2,
                     max_length=2))
results

[]

In [9]:
#데이터프레임 정리
columns=['source', 'target','support']
network_df=pd.DataFrame(columns=columns)

#규칙의 조건절을 source, 결과절을 target, 지지도를 support 라는 데이터 프레임의 피처로 변환
for result in results:
    if len(result.items)==2:
        items=[x for x in result.items]
        row=[items[0], items[1], result.support]
        series = pd.Series(row, index=network_df.colud)

In [12]:
#말뭉치 추출
corpus=''.join(df['제목'].tolist())
print(corpus)
from konlpy.tag import Okt
from collections import Counter

nouns_tagger=Okt()
nouns=nouns_tagger.nouns(corpus)
count=Counter(nouns)

remove_char_counter = Counter({x:count[x] for x in count})

스쿨존불법오토바이건설행정과직원들의불합리한단속을신고합니다동명동인도공사인가주차장공사인가동명동인도공사인가주차장공사인가동명동인도주차위험지적재조사건축과불법건축담당은답변을달면문자바랍니다청장님과비서실장에게얘기한것입니다친절하고자상한보건소방문보건계직원학교앞무허가하우스동명동보도블럭불법주차충장로가상가지역화분지원동주민센터옆공사인도길막음해결해주세오동명동부리나치킨삼거리신호등신설요망건축과이현승님조대후문일방통행로위험합니다공사로인해인도보행불가합니다동명로도로에관하여충장로가엔터빌딩맞은편건물내공사증심사길블라드교체딱한군데만빼놓은이유를알고싶습니다음식조리사조건유예근로능력재판정중조건유예근로능력재판정중전신주단자함열림급질의건축과철거관련과조수연주무관님칭찬합니다동명동인도공사내집앞주차지역표시청년체육과이수일계장님정호엽주무관님감사합니다칭찬합니다조대후문살레시오여고앞너무위험합니다보안등고장입니다주정차위반단속을경찰로이관바랍니다건축과이현승님께요청합니다제글비밀번호가바꼈네요소태동태광시프드앞주차정비요망충장축제방역불법건축물신고같은내용번째몇번신고하면처리하나요조건유예근로능력재판정중개념없는애완견주인배변방법없을까요살레오시오여고옆일방통행로불법주정차충장축제해도해도너무하네요보안등고장동명동인도공사주차장동명동거리인도설치후주차문제아파트층상가편의점앞불법적으로테이블의자천막을처놓고영업을하고있습니다보행자가안전하게통행할수있게제발불법적치물수거해가주세요살레시오여고옆오토바이소음푸른길잡상인어찌연락이없으시네요가설문철거가로등고장이요식당인지주차장인지조대후문살레시오여고일방통행로위험합니다계림교회십자가빛공해가너무심합니다재개발진행요청건도시텃밭분양신청은언제받나요주차장문제노인일자리정다운교육조대후문살레시오여고옆위험합니다주차장영업맨홀뚜껑덜컹거림가로등고장요퇴근시간때월남동쪽순환입구교통정리해주세요지산동조대후문오토바이매우위험합니다지산동원룸근처개짖는소리로인한소음지산동조대후문오토바이들너무위험합니다돌출간판이위험합니다감사지적이행사항확인요망지산동정류장일대모기떼피해가구다차량주차에따른수익주차장행단보도입구바닥보수좀해주세용삼총사부동산과해드림부동산이면도로황색주차선주변건물리모델링시소음및분진문제계도요청건동명동통

In [13]:
node_df=pd.DataFrame(remove_char_counter.items(), columns=['node', 'nodesize'])
node_df=node_df[node_df['nodesize']>5]
node_df

,node,nodesize
1,불법,193
2,오토바이,17
3,건설,46
4,행정,31
5,직원,20
...,...,...
1800,반대,6
1863,대표,7
1936,이마트,11
2092,오펠,6


In [14]:
!pip install networkx

# networkx
import matplotlib.pyplot as plt
import networkx as nx
plt.figure(figsize=(25,25))

G=nx.Graph()
for index, row in node_df.iterrows():
    G.add_node(row['node'], nodesize=row['nodesize'])

for index, row in network_df.iterrows():
    G.add_weighted_edges_from([row['source'], row['target'], row['support']])
    

    

<Figure size 1800x1800 with 0 Axes>